In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
np.zeros((4,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [37]:
# Variables, parameters, initializations
num_simulations = 1000
N = 1000 # true population size
num_gens = 300
num_pops = 2 # pop A and pop B

# Store 2-D lists for the Ne, SDI, and number of unique variants for each population PER generation
ne_mtx = np.zeros((num_gens, num_pops))
sdi_mtx = np.zeros((num_gens, num_pops))
unique_vars = np.zeros((num_gens, num_pops))

mu = 10 ** -3 # innovation rate
theta = 1
role_model_prop = 0.5




In [ ]:
def Main():
    for i in range proportion of role models:
        for j in range of innovation rates and each theta:
            for n in range (num_simulations):
                RunSimulation()

In [ ]:
def RunSimulation():
    RunBurnIn()
    for gen in range num_gens():
        RunGeneration()
    

In [ ]:
def RunBurnIn():
    initialize pop a and pop b sizes and variants distribution
    sdi_a = CalcSimpsonsIndex(pop_a)
    sdi_b = CalcSimposonsIndex(pop_b)
    while sdi_a > sdi_b:
        perform cultural transmission
        do innovation
        keep track of new cultural variants per pop
        update pop_a and pop_b
        CalcSimpsonsIndex(pop_a)
        CalcSimpsonsIndex(pop_b)
    store and return 2d lists for Ne, SDI, unique num variants...

In [ ]:
def RunGeneration():
    cultural_trans_a = CalcCulturalTransmission(pop_a)
    cultural_trans_b = CalcCulturalTransmission(pop_b)
    Assign new variants to pop
    Calc k_is from new variants
    Calc Nes, SDI, number of unique variants
    Undergo Innovation
    Keep track of new total number of unique variants after innovators added
    Store Nes, SDI, num unique traits before innovation, and parameters (prop of role models, theta, gen num)

In [ ]:
def CalcSimpsonsIndex(unique_variants_list, N):
    simpsons_sum = 0
    for i in range unique_variants_list:
        simpsons_sum += ((unique_variants_list[i] / N) ** 2)
    simpsons_index = 1 - simpsons_sum
    return simpsons_index

In [ ]:
def CalcNumUniqueVariants():

In [ ]:
def CalcEffectivePop(N, variance):
    return (N-1)/variance

In [ ]:
def CalcCulturalTransmission():

In [ ]:
def PerformCulturalTransmission(pop_a, pop_b):
    randomly introduce new variants for pop_a and pop_b

In [ ]:
def PerformInnovation(N, mu):
    np.random.binomial(N, mu)

In [36]:
np.random.binomial(N, mu)

0